In [23]:
import pandas as pd
import numpy as np
import mysql.connector as mysql
import time
from datetime import date, timedelta
import re 
import os

query0 = {'approved_suburbs' : f"""SELECT name 
                            FROM look_up.approved_ssc"""}
query0 = {k: " ".join(query0[k].replace(f"\n","").split()) for k, v in query0.items()}

elderbrain_connection = mysql.connect(
        host="45.76.126.137",
        user="alvin",
        passwd="xeLGLOVI1ZtSMhW2"
        ) 

res = []
sb = []
for sql in query0:
    elderbrain_cursor0 = elderbrain_connection.cursor()
    elderbrain_cursor0.execute(query0[sql])
    data0 = elderbrain_cursor0.fetchall()

# remove content within parentheses
for r in data0:
    res.append(r[0])
    
# add outputs back to the list
for elem in res:
    sb.append(elem)
    
# make variables
approved_suburbs = tuple(sb)
today = date.today()
previous_date = date.today() - timedelta(1)

query = {'todays_listings' : f"""SELECT u_id,
                            first_date,
                            status,
                            address,                    
                            TRIM(REGEXP_SUBSTR(suburb,"[A-z ]*")) AS suburb,
                            price,
                            property_type,
                            bedrooms,
                            bathrooms,
                            land_size
                            FROM real_estate.buy_campaign
                            WHERE first_date BETWEEN '{previous_date}' AND '{today}' AND property_type = 'house' AND suburb in {approved_suburbs} AND status IS NULL AND (land_size >= '400' OR land_size IS NULL)
                            ORDER BY suburb"""
        }
query = {k: " ".join(query[k].replace(f"\n","").split()) for k, v in query.items()}


elderbrain_cursor = elderbrain_connection.cursor()

for sql in query:
        print(f"Collecting {sql}...", end =" ")
        start = time.time()
        elderbrain_cursor.execute(query[sql])
        data = elderbrain_cursor.fetchall()
        colnames = [desc[0] for desc in elderbrain_cursor.description] # get a list of column names
        data = pd.DataFrame(data,columns=colnames)
        query[sql] = data 
        end = time.time()
        print(f"Done! It took {round(end - start)} seconds. Good luck!") 
        
        
elderbrain_connection.close()

df = pd.DataFrame(query[sql])

          u_id  first_date status  \
0    138871351  2022-03-22   None   
1    138871371  2022-03-22   None   
2    138742187  2022-03-22   None   
3    138883763  2022-03-23   None   
4    138622943  2022-03-22   None   
..         ...         ...    ...   
221  138871507  2022-03-22   None   
222  138562235  2022-03-22   None   
223  138562235  2022-03-23   None   
224  138779167  2022-03-22   None   
225  138260514  2022-03-23   None   

                                           address           suburb  \
0          120 Congewai Street, Aberdare, NSW 2325         Aberdare   
1         10 Devon Road, Alexandra Hills, Qld 4161  Alexandra Hills   
2       9 Jasper Street, Alexandra Hills, Qld 4161  Alexandra Hills   
3       19 Palana Drive, Alexandra Hills, Qld 4161  Alexandra Hills   
4    17 Greystoke Place, Alexandra Hills, Qld 4161  Alexandra Hills   
..                                             ...              ...   
221          11 Anglesey Street, Wodonga, Vic 3690        

In [28]:
# separate & clean address
new = df.copy()
new = new[new['address'].notna()]
new = new[~new.address.str.contains('Lot')]
new = new[~new.address.str.contains('/')]
new[['address','suburb','newcl','x']] = new.address.str.split(',', expand=True)
new[['state', 'postcode']] = new.newcl.str.split(" ", expand=True, n=1)
new = new.drop('state', axis = 1)
new[['state', 'num']] = new.postcode.str.split(" ", expand=True)
new = new.drop('num', axis = 1)
new = new.drop('postcode', axis = 1)
new = new.drop('newcl', axis = 1)
new = new.reindex(columns=['u_id', 'first_date', 'status', 'address', 'suburb', 'state', 'price', 'property_type', 'bedrooms', 'bathrooms', 'land_size'])
print('address formatted')
print(new)

address formatted
          u_id  first_date status              address            suburb  \
0    138871351  2022-03-22   None  120 Congewai Street          Aberdare   
1    138871371  2022-03-22   None        10 Devon Road   Alexandra Hills   
2    138742187  2022-03-22   None      9 Jasper Street   Alexandra Hills   
3    138883763  2022-03-23   None      19 Palana Drive   Alexandra Hills   
4    138622943  2022-03-22   None   17 Greystoke Place   Alexandra Hills   
..         ...         ...    ...                  ...               ...   
221  138871507  2022-03-22   None   11 Anglesey Street           Wodonga   
222  138562235  2022-03-22   None   41 Blackall Street           Woombye   
223  138562235  2022-03-23   None   41 Blackall Street           Woombye   
224  138779167  2022-03-22   None    26 Minori Gardens          Yangebup   
225  138260514  2022-03-23   None    18B Scala Gardens          Yangebup   

    state                         price property_type bedrooms bathro

In [31]:
# make a url
# variables
# loop through each variable

def create_url(suburb, property_type, state, u_id):
    url = f"https://www.realestate.com.au/property-{suburb}-{property_type}-{state}-{u_id}"
    url = url.replace(f" ","")
    return url

L = []
new.copy()
for row in new.index:
     sub = new['suburb'][row]
     prt = new['property_type'][row]
     sta = new['state'][row]
     uid = new['u_id'][row]
     link = create_url(sub, prt, sta, uid)
    #convert string generated by loop into a df
     L.append(link)

In [32]:
try: 
    new = new.drop(['u_id', 'status'], axis = 1)
    new.insert(9, 'urllink', L)
    new['address'].replace('', np.nan, inplace=True)
    new.dropna(subset=['suburb'], inplace=True)
    new = new[~new['address'].str.contains('\/')]
    print('Table generated')
except ValueError:
    print('urllink already exists, there is no need to do it again!')
    pass

Table generated


In [33]:
def fun(path):
    
    # returns the final component of a path
    f_url = os.path.basename(path)
      
    # convert the path into clickable link
    return '<a href="{}">{}</a>'.format(path, f_url)

new = new.style.format({'urllink': fun})  
print('urllink activated successfully')

urllink activated successfully


In [34]:
try:
    new.to_excel('C:/Users/AlvinXXX/results.xlsx', index = False)  
    print('listings saved successfully')
except PermissionError:
    print('Hey bro, fail to save it. Please close the spreadsheet first and then try again')
    pass

listings saved successfully
